# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

# <font color='blue'>Capítulo 8</font>

### *********** Atenção: *********** 
Utilize Java JDK 1.8 e Apache Spark 2.4.2

Java JDK 1.8:

https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html

*Caso receba mensagem de erro "name 'sc' is not defined", interrompa o pyspark e apague o diretório metastore_db no mesmo diretório onde está este Jupyter notebook*

# Spark SQL

O Spark SQL é usado para acessar dados estruturados com Spark.

#### Acesse http://localhost:4040 sempre que quiser acompanhar a execução dos jobs. 
#### Pacotes adicionais podem ser encontrados aqui: https://spark-packages.org/ (usaremos um destes pacotes para conexão com o MongoDB).

## Spark SQL - Spark Session e SQL Context

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [ ]:
print(sc)

In [ ]:
# Spark Session - usado para trabalhar com o Spark
spSession = SparkSession.builder.master("local").appName("DSA-SparkSQL").getOrCreate()

In [ ]:
# Criando o SQL Context para trabalhar com Spark SQL
sqlContext = SQLContext(sc)

In [ ]:
# Importando o arquivo e criando um RDD
linhasRDD1 = sc.textFile("data/carros.csv")

In [ ]:
linhasRDD1.count()

In [ ]:
# Removendo a primeira linha - Transformação 1
linhasRDD2 = linhasRDD1.filter(lambda x: "FUELTYPE" not in x)

In [ ]:
linhasRDD2.count()

In [ ]:
# Dividindo o conjunto de dados em colunas - Transformação 2
linhasRDD3 = linhasRDD2.map(lambda line: line.split(","))

In [ ]:
# Dividindo o conjunto de dados em colunas - Transformação 3
linhasRDD4 = linhasRDD3.map(lambda p: Row(make = p[0], body = p[4], hp = int(p[7])))

In [ ]:
print(linhasRDD4)

In [ ]:
?Row

In [ ]:
linhasRDD4.collect()

In [ ]:
# Criando um dataframe a partir do RDD
linhasDF = spSession.createDataFrame(linhasRDD4)

In [ ]:
linhasDF.show()

In [ ]:
type(linhasDF)

In [ ]:
# Mesma coisa que: SELECT * FROM linhasDF
linhasDF.select("*").show()

In [ ]:
# Mesma coisa que: SELECT * FROM linhasDF ORDER BY make
linhasDF.orderBy("make").show()

In [ ]:
# Registrando o dataframe como uma Temp Table
linhasDF.createOrReplaceTempView("linhasTB")

In [ ]:
!java -version

In [ ]:
# Executando queries SQL ANSI
spSession.sql("select * from linhasTB where make = 'nissan'").show()

In [ ]:
# Executando queries SQL ANSI
spSession.sql("select make, body, avg(hp) from linhasTB group by make, body").show()

## Spark SQL e Arquivos CSV

In [ ]:
carrosDF = spSession.read.csv("data/carros.csv", header = True)

In [ ]:
type(carrosDF)

In [ ]:
carrosDF.show()

In [ ]:
# Registrando o dataframe como uma Temp Table
carrosDF.createOrReplaceTempView("carrosTB")

In [ ]:
# Executando queries SQL ANSI
spSession.sql("select make, hp, price from carrosTB where CYLINDERS = 'three'").show()

In [ ]:
carrosTT = spSession.sql("select make, hp, price from carrosTB where CYLINDERS = 'three'")

In [ ]:
carrosTT.show()

## Aplicando Machine Learning

In [ ]:
# Carregando o arquivo CSV e mantendo o objeto em cache
carros = sc.textFile("data/carros.csv")
carros.cache()

In [ ]:
# Remove a primeira linha (header)
primeiraLinha = carros.first()
linhas = carros.filter(lambda x: x != primeiraLinha)
linhas.count()

In [ ]:
# Importando função row
from pyspark.sql import Row

In [ ]:
# Convertendo para um vetor de linhas
def transformToNumeric(inputStr) :
    
    attList = inputStr.split(",")
    
    doors = 1.0 if attList[3] == "two" else 2.0
    
    body = 1.0 if attList[4] == "sedan" else 2.0 
       
    # Filtrando colunas não necessárias nesta etapa
    valores = Row(DOORS = doors, BODY = float(body), HP = float(attList[7]), RPM = float(attList[8]), MPG = float(attList[9]))
    return valores

In [ ]:
# Aplicando a função aos dados e persistindo o resultado em memória
autoMap = linhas.map(transformToNumeric)
autoMap.persist()
autoMap.collect()

In [ ]:
# Criando o Dataframe
carrosDf = spSession.createDataFrame(autoMap)
carrosDf.show()

In [ ]:
# Sumarizando as estatísticas do conjunto de dados
summStats = carrosDf.describe().toPandas()
summStats

In [ ]:
# Extraindo as médias
medias = summStats.iloc[1,1:5].values.tolist()
medias

In [ ]:
# Extraindo o desvio padrão
desvios_padroes = summStats.iloc[2,1:5].values.tolist()
desvios_padroes

In [ ]:
# Inserindo a média e o desvio padrão em uma variável do tipo broadcast 
bcMedias = sc.broadcast(medias)
bcDesviosP = sc.broadcast(desvios_padroes)

In [ ]:
# Importando a Função Vectors
from pyspark.ml.linalg import Vectors

In [ ]:
# Função para normalizar os dados e criar um vetor denso
def centerAndScale(inRow) :
    global bcMedias
    global bcDesviosP
    
    meanArray = bcMedias.value
    stdArray = bcDesviosP.value

    retArray = []
    
    for i in range(len(meanArray)):
        retArray.append( (float(inRow[i]) - float(meanArray[i])) / float(stdArray[i]) )
    return Vectors.dense(retArray)

In [ ]:
# Aplicando a normalização aos dados
csAuto = carrosDf.rdd.map(centerAndScale)
csAuto.collect()

In [ ]:
# Criando um Spark Dataframe com as features (atributos)
autoRows = csAuto.map(lambda f: Row(features = f))
autoDf = spSession.createDataFrame(autoRows)
autoDf.select("features").show(10)

In [ ]:
# Importando o algoritmo K-Means para clusterização
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k = 3, seed = 1)
modelo = kmeans.fit(autoDf)
previsoes = modelo.transform(autoDf)
previsoes.show()

In [ ]:
# Plot dos resultados
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
# Função para leitura dos dados e plotagem
def unstripData(instr) :
    return ( instr["prediction"], instr["features"][0], instr["features"][1],instr["features"][2],instr["features"][3])


In [ ]:
# Organizando os dados para o Plot
unstripped = previsoes.rdd.map(unstripData)
predList = unstripped.collect()
predPd = pd.DataFrame(predList)

In [ ]:
plt.cla()
plt.scatter(predPd[3], predPd[4], c = predPd[0])

## Spark SQL e Arquivos JSON

Neste site você pode validar a estrutura de um arquivo JSON: http://jsonlint.com/

In [ ]:
# Importando o arquivo JSON
funcDF = spSession.read.json("data/funcionarios.json")

In [ ]:
funcDF.show()

In [ ]:
funcDF.printSchema()

In [ ]:
type(funcDF)

In [ ]:
# Operações com Dataframe Spark SQL - select()
funcDF.select("nome").show()

In [ ]:
# Operações com Dataframe Spark SQL - filter()
funcDF.filter(funcDF["idade"] == 50).show()

In [ ]:
# Operações com Dataframe Spark SQL - groupBy()
funcDF.groupBy("sexo").count().show()

In [ ]:
# Operações com Dataframe Spark SQL - groupBy()
funcDF.groupBy("deptid").agg({"salario": "avg", "idade": "max"}).show()

In [ ]:
# Registrando o dataframe como uma Temp Table
funcDF.registerTempTable("funcTB")

In [ ]:
# Executando queries SQL ANSI
spSession.sql("select deptid, max(idade), avg(salario) from funcTB group by deptid").show()

## Temp Tables

In [ ]:
# Registrando o dataframe como temp Table
funcDF.createOrReplaceTempView("funcTB")

In [ ]:
spSession.sql("select * from funcTB where salario = 9700").show()

In [ ]:
# Criando Temp Table
sqlContext.registerDataFrameAsTable(funcDF, "funcTB2")

In [ ]:
type(funcTB2)

In [ ]:
# Persistindo a Temp Table 
funcTB3 = spSession.table("funcTB2")

In [ ]:
type(funcTB3)

In [ ]:
# Comparando o Dataframe com a tabela temporária criada
sorted(funcDF.collect()) == sorted(funcTB3.collect())

In [ ]:
# Aplicando o filtro
sqlContext.registerDataFrameAsTable(funcDF, "funcTB2")
funcTB3 = spSession.table("funcTB2")
funcTB3.filter("idade = '42'").first()

In [ ]:
# Drop Temp Table
sqlContext.dropTempTable("funcTB2")

## Banco de Dados Relacional

Extraindo Dados do MySQL. Primeiro precisamos baixar o driver JDBC. Haverá um driver JDBC para cada banco de dados que você conectar (Oracle, SQL Server, etc...)

1- Download do Driver JDBC para o MySQL: http://dev.mysql.com/downloads/connector/j/

2- Baixar o arquivo .zip

3- Descompactar o arquivo e copiar o arquivo mysql-connector-java-8.0.16.jar para a pasta /opt/Spark/jars ou para SO Windows em C:\Spark\jars

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spSession = SparkSession.builder.master("local").appName("DSA-SparkSQL").getOrCreate()
sqlContext = SQLContext(sc)

In [ ]:
mysql_df = spSession.read.format("jdbc").options(
    url = "jdbc:mysql://localhost/carros",
    serverTimezone = "UTC",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "carrosTB",
    user = "root",
    password = "dsa1234").load()

In [ ]:
mysql_df.show()

In [ ]:
mysql_df.registerTempTable("carrostb")

In [ ]:
spSession.sql("select * from carrostb where hp = '68'").show()

## Banco de Dados Não-Relacional

Spark Connector: https://docs.mongodb.com/spark-connector/current/

Mongo Spark: https://spark-packages.org/package/mongodb/mongo-spark

$SPARK_HOME/bin/pyspark --packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0

In [ ]:
# Imports
from pyspark.sql import SparkSession

### Leitura

In [ ]:
# Cria a sessão
my_spark = SparkSession \
    .builder \
    .appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://localhost/test_db.test_collection") \
    .config("spark.mongodb.output.uri", "mongodb://localhost/test_db.test_collection") \
    .getOrCreate()

In [ ]:
# Carrega os dados do MongoDB no Spark
dados = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [ ]:
dados.printSchema()

In [ ]:
dados.count()

In [ ]:
dados.head()

In [ ]:
dados.show()

### Gravação

In [ ]:
registro = spark.createDataFrame([("Camisa T-Shirt",  50)], ["item", "qty"])

In [ ]:
registro.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()

In [ ]:
dados.show()

# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>